In [1]:
import soundata
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from IPython.display import Audio

import librosa
import torchlibrosa
import tqdm
import models

In [2]:
class MixAugmentDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, signal_categories, noise_categories, folds=[1,2,3], augment_type=None, sample_rate=22050, 
                    window_size=2048, mel_bins=128, hop_size=512, fmin=50, fmax=11025):
        self.augment_type = augment_type
        self.sample_rate = sample_rate
        
        x = []
        y = []
        noise = []
        clips = esc50.load_clips()
        
        for clip_id in tqdm.tqdm(dataset.clip_ids):
            clip = clips[clip_id]
            if clip.fold in folds:
                waveform = clip.audio[0]
                original_sample_rate = clip.audio[1]
                
                waveform = librosa.resample(
                    waveform, 
                    original_sample_rate, 
                    sample_rate,
                    res_type='kaiser_fast'
                )
                
                self.sample_rate = clip.audio[1]
                
                if clip.category in noise_categories:
                    noise.append(waveform)
                elif clip.category in signal_categories:
                    x.append(waveform)
                    # set label to the index of the signal category
                    label = np.where(signal_categories == clip.category)[0][0]
                    y.append(label)
                    
        self.x = np.array(x)
        self.y = np.array(y)
        self.noise = np.array(noise)
        
        # feature extractors   
        def logmel_extractor(z):
            return librosa.feature.melspectrogram(
                y          = z, 
                sr         = self.sample_rate,
                n_mels     = mel_bins,
                n_fft      = window_size, 
                hop_length = hop_size, 
                win_length = None, 
                window     = 'hann', 
                center     = True, 
                pad_mode   = 'reflect', 
                power      = 2.0,
                fmin       = fmin,
                fmax       = fmax,
                #ref        = 1.0,
                #amin       = 1e-10,
                #top_db     = None
            )
        
        self.logmel_extractor = logmel_extractor
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        
        if self.augment_type == 'waveform':
            p = np.random.uniform(0.1, 0.3)
            idx = np.random.randint(0, len(self.noise))
            aug = self.noise[idx]
            x_aug = (x * (1-p)) + (aug * p)
            
            # extract logmel after augmentation
            x_aug = self.logmel_extractor(x_aug)
            x_aug = np.transpose(x_aug, (1, 0))
            x_aug = np.expand_dims(x_aug, 0)
            
        elif self.augment_type == 'logmel':
            p = np.random.uniform(0.1, 0.3)
            idx = np.random.randint(0, len(self.noise))
            aug = self.noise[idx]
            
            # extract logmel before augmentation
            x = self.logmel_extractor(x)
            x = np.transpose(x, (1, 0))
            x = np.expand_dims(x, 0)
            
            aug = self.logmel_extractor(aug)
            aug = np.transpose(aug, (1, 0))
            aug = np.expand_dims(aug, 0)
            
            x_aug = (x * (1-p)) + (aug * p)
        else:
            # extract logmel without augmentation
            x_aug = self.logmel_extractor(x)
            x_aug = np.transpose(x_aug, (1, 0))
            x_aug = np.expand_dims(x_aug, 0)
        return x_aug, y

In [3]:
def random_signal_and_noise_categories(dataset, nb_signal_categories):
    categories = []
    clips = dataset.load_clips()
    clip_ids = dataset.clip_ids
    for clip_id in clip_ids:
        categories.append(clips[clip_id].category)
    categories = list(set(categories))

    signal_categories = np.random.choice(categories, nb_signal_categories, replace=False)
    noise_categories = list(set(categories).difference(set(signal_categories)))
    
    return signal_categories, noise_categories

In [4]:
def train(model, optimizer, loss_function, train_loader):
    model.train()
    
    running_loss = 0
    count = 0
    for (x, y) in tqdm.tqdm(train_loader):
        x = x.cuda()
        y = y.type(torch.LongTensor).cuda()
        
        optimizer.zero_grad()
        
        y_pred = model(x)['clipwise_output']
        loss = loss_function(y_pred, y)
        loss.backward()
        
        optimizer.step()
    
        running_loss += loss.item()
        count += 1
    return running_loss / count

In [5]:
def evaluate(model, loss_function, loader):
    model.eval()
    
    count = 0
    running_acc = 0
    running_loss = 0
    
    ys = []
    ys_pred_probs = []
    for (x, y) in loader:
        x = x.cuda()
        y = y.type(torch.LongTensor).cuda()
        
        y_pred = model(x)['clipwise_output']
        loss = loss_function(y_pred, y)
        running_loss += loss.item()
        
        y_pred_prob = y_pred.detach().cpu().numpy()
        y_pred = np.argmax(y_pred_prob, axis=1)
        y      = y.detach().cpu().numpy().astype(np.int)

        running_acc += np.mean(y==y_pred)
        
        count+=1
        
        ys.append(y)
        ys_pred_probs.append(y_pred_prob)
    
    return running_loss / count, running_acc / count, np.concatenate(ys), np.concatenate(ys_pred_probs)

In [ ]:
# setup writer
writer = SummaryWriter(log_dir='log_dir/no_augmentation')

# setup model
sample_rate = 22050
window_size = 2048
hop_size = 512
mel_bins = 128
fmin = 50
fmax = sample_rate // 2
classes_num = 10
feature_type = 'logmel'

model = models.Cnn6(sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num, feature_type)
model = model.cuda()

best_model = models.Cnn6(sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num, feature_type)
best_model = best_model.cuda()

# setup training
learning_rate = 1e-4
patience = 100
epochs = 1000

optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

augment_type = None

# setup datasets
esc50 = soundata.initialize('esc50')
train_folds = [1,2,3]
valid_folds = [4]
test_folds = [5]

signal_categories, noise_categories = random_signal_and_noise_categories(esc50, classes_num)

train_dataset = MixAugmentDataset(esc50, signal_categories, noise_categories, folds=train_folds, 
                                 augment_type=augment_type, sample_rate=sample_rate,
                                 window_size=window_size, mel_bins=mel_bins, hop_size=hop_size,
                                 fmin=fmin, fmax=fmax)
valid_dataset = MixAugmentDataset(esc50, signal_categories, noise_categories, folds=valid_folds, 
                                 augment_type=augment_type, sample_rate=sample_rate,
                                 window_size=window_size, mel_bins=mel_bins, hop_size=hop_size,
                                 fmin=fmin, fmax=fmax)
# test_dataset = MixAugmentDataset(esc50, signal_categories, noise_categories, folds=test_folds, 
#                                 augment=augment, sample_rate=sample_rate,
#                                 window_size=window_size, mel_bins=mel_bins, hop_size=hop_size,
#                                 fmin=fmin, fmax=fmax)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=8)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=8)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)


best_valid_loss = np.inf
best_epoch = 0
epoch = 0
not_converged = True
while not_converged:
    print("Epoch: {}".format(epoch))
    train_loss = train(model, optimizer, loss_function, train_loader)
    valid_loss, valid_acc, _, _ = evaluate(model, loss_function, valid_loader)
    print("valid loss: {}, acc: {}".format(valid_loss, valid_acc))
    writer.add_scalar('loss/train', train_loss, epoch)
    writer.add_scalar('loss/valid', valid_loss, epoch)
    writer.add_scalar('acc/valid', valid_acc, epoch)

    epoch += 1

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_epoch = epoch
        best_model.load_state_dict(model.state_dict())

    # convergence criterion
    if epoch - best_epoch >= patience or epoch > epochs:
        not_converged = False

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 0


  0%|          | 0/15 [00:00<?, ?it/s]

valid loss: 2.2728600025177004, acc: 0.15
Epoch: 1


  0%|          | 0/15 [00:00<?, ?it/s]

...

valid loss: 2.183646631240845, acc: 0.1625
Epoch: 37


  0%|          | 0/15 [00:00<?, ?it/s]

valid loss: 2.1265883922576903, acc: 0.275
Epoch: 38


  0%|          | 0/15 [00:00<?, ?it/s]

valid loss: 2.0673206090927123, acc: 0.325
Epoch: 39
